In [ ]:
!pip install opencv-python
!pip install opencv-contrib-python

!pip install ffmpeg moviepy

from google.colab import drive
drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import sys
import cv2
import numpy as np

import math
import moviepy.editor as mp
from operator import itemgetter

In [ ]:
def find_speaking(audio_clip, window_size=20, volume_threshold=0.0001):

    num_windows = math.floor(audio_clip.end/window_size)

    window_is_silent = []
    for i in range(num_windows):
        s = audio_clip.subclip(i * window_size, (i + 1) * window_size)

        v = s.max_volume()
        window_is_silent.append(v < volume_threshold)

    start = 0
    end = 0
    speaking_intervals = []
    for i in range(1, len(window_is_silent)):
        if window_is_silent[i - 1] and not window_is_silent[i]:
            start = i * window_size

        if not window_is_silent[i - 1] and window_is_silent[i]:
            end = i * window_size
            new_speaking_interval = [start - 0.25, end + 0.25]

            need_to_merge = len(speaking_intervals) > 0 and speaking_intervals[-1][1] > new_speaking_interval[0]
            if need_to_merge:
                merged_interval = [speaking_intervals[-1][0], new_speaking_interval[1]]
                speaking_intervals[-1] = merged_interval
            else:
                element = (new_speaking_interval, new_speaking_interval[1] - new_speaking_interval[0])
                speaking_intervals.append(element)

    return speaking_intervals

In [ ]:
def find_longest_speaking_interval(speaking_intervals):    
    maximum = max(speaking_intervals, key=itemgetter(1))[0] 
    return maximum 

In [ ]:
clip = mp.VideoFileClip('drive/MyDrive/dataset/2022-04-11 14_31 Software Design, лекция, ВШЭ3, весна 2022.mp4')
audio = clip.audio

In [ ]:
  # longest silence interval in milliseconds
result = find_longest_speaking_interval(find_speaking(audio))

In [ ]:
print(result)

[839.75, 5580.25]


In [ ]:
def find_shot_transition_start(cap):
    threshold = 125

  # average intensity of pixels in a frame
    last_mean = 0 
    start_lection = 0

    while True:
      #  get the image of current frame
        (rv, im) = cap.read()
        # if cant get the frame -- break
        if not rv:
            break
        # get mean intensivity of pixels (colors in rgb)
        frame_mean = np.sum(im) / float(im.shape[0] * im.shape[1] * im.shape[2])

        # check if we reach the start timecode from silence detection
        if cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0 >= result[0]:
            #  return silence timecode if we not detect shot transition
            if start_lection == 0:
                return result[0]
            break
        # check changes of mean intensivity, аnd save timecode
        if frame_mean >= threshold and last_mean < threshold:
            start_lection = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0

        elif frame_mean < threshold and last_mean >= threshold:
            start_lection = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0

        last_mean = frame_mean

    return start_lection

In [ ]:
def find_shot_transition_end(cap):
    threshold = 125
  # average intensity of pixels in a frame
    last_mean = 0 

  # video number of frames (for getting last index) and fps length in seconds
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT) - 2

    end_lection = 0

    while True:
        # set the frame to the video
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_count)
        (rv, im) = cap.read()
        # run through frames in 4 times fps for quick processing
        frame_count = frame_count - 4 * fps

        if not rv:
            break

        frame_mean = np.sum(im) / float(im.shape[0] * im.shape[1] * im.shape[2])

        if cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0 <= result[1]:
            if end_lection == 0:
                return result[1]
            break

        if frame_mean >= threshold and last_mean < threshold:
            end_lection = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0

        elif frame_mean < threshold and last_mean >= threshold:
            end_lection = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0

        last_mean = frame_mean
        
    return end_lection

In [ ]:
cap = cv2.VideoCapture('drive/MyDrive/dataset/2022-04-11 14_31 Software Design, лекция, ВШЭ3, весна 2022.mp4')

In [ ]:

correct_answer = [find_shot_transition_start(cap), find_shot_transition_end(cap)]
cap.release()

In [ ]:
print(correct_answer)

(580.32, 5580.25)
